# Examples Of Querying Online Data

I have a few helper functions for querying ESRI and Socrata datasets.

In [1]:
from crimepy import query
import requests
from urllib.parse import urlencode
import pandas as pd

# EXAMPLE SOCRATA

# Chicago Robberies in the past 30 days
last30 = (pd.to_datetime('now') - pd.Timedelta(30,'d')).strftime('%Y-%m-%d')
print(f'Date for last 30 days, {last30}')
chic_url = 'https://data.cityofchicago.org/resource/ijzp-q8t2.json' # can get this from Socrata page
parm = f"?$where=date>'{last30}'&primary_type='ROBBERY'" # only selecting out robberies
res_rob = query.query_socrata(chic_url,add_params=parm)
res_rob

Date for last 30 days, 2025-08-08


,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,13932366,JJ372512,2025-08-08T00:30:00.000,060XX S GREEN ST,0320,ROBBERY,STRONG ARM - NO WEAPON,STREET,False,True,...,16,68,03,1171674,1864781,2025,2025-08-16T15:48:00.000,41.784416556,-87.646116386,"{'latitude': '41.784416556', 'longitude': '-87..."
1,13927633,JJ366506,2025-08-08T01:24:00.000,005XX S WELLS ST,0325,ROBBERY,VEHICULAR HIJACKING,STREET,False,False,...,34,32,03,1174804,1897931,2025,2025-08-16T15:48:00.000,41.875313758,-87.633650701,"{'latitude': '41.875313758', 'longitude': '-87..."
2,13927887,JJ366886,2025-08-08T11:56:00.000,039XX W MADISON ST,031A,ROBBERY,ARMED - HANDGUN,RESTAURANT,False,False,...,28,26,03,1150223,1899725,2025,2025-08-16T15:48:00.000,41.880751096,-87.723855981,"{'latitude': '41.880751096', 'longitude': '-87..."
3,13928206,JJ367160,2025-08-08T14:30:00.000,027XX W MADISON ST,0330,ROBBERY,AGGRAVATED,ALLEY,False,False,...,27,27,03,1158054,1899904,2025,2025-08-16T15:48:00.000,41.881086082,-87.695096188,"{'latitude': '41.881086082', 'longitude': '-87..."
4,13928141,JJ367189,2025-08-08T14:30:00.000,059XX W SUPERIOR ST,0326,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,STREET,False,False,...,29,25,03,1136628,1904394,2025,2025-08-16T15:48:00.000,41.893817529,-87.77366461,"{'latitude': '41.893817529', 'longitude': '-87..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,13952124,JJ396701,2025-08-29T21:00:00.000,003XX W 75TH ST,031A,ROBBERY,ARMED - HANDGUN,STREET,False,False,...,6,69,03,1175338,1855221,2025,2025-09-06T15:53:18.000,41.758101695,-87.632968191,"{'latitude': '41.758101695', 'longitude': '-87..."
400,13950753,JJ394982,2025-08-29T21:00:00.000,008XX N LOCKWOOD AVE,031A,ROBBERY,ARMED - HANDGUN,STREET,False,False,...,37,25,03,1140872,1904987,2025,2025-09-06T15:53:18.000,41.895367769,-87.75806298,"{'latitude': '41.895367769', 'longitude': '-87..."
401,13950597,JJ394777,2025-08-29T21:45:00.000,0000X N MENARD AVE,031A,ROBBERY,ARMED - HANDGUN,SIDEWALK,False,False,...,29,25,03,1137713,1899777,2025,2025-09-06T15:53:18.000,41.881128394,-87.769791062,"{'latitude': '41.881128394', 'longitude': '-87..."
402,13954583,JJ399704,2025-08-29T22:00:00.000,055XX W ROOSEVELT RD,0320,ROBBERY,STRONG ARM - NO WEAPON,PARKING LOT / GARAGE (NON RESIDENTIAL),False,False,...,29,25,03,1139424,1894135,2025,2025-09-06T15:53:18.000,41.865614992,-87.763645777,"{'latitude': '41.865614992', 'longitude': '-87..."


In [2]:
# Example ESRI queries
# this will page through long results

# DC getting entire dataset for a year
# they have years in different URLS, see https://maps2.dcgis.dc.gov/dcgis/rest/services/FEEDS/MPD/MapServer

year_id = {2008: '32',
           2009: '33',
           2010: '34',
           2011: '35',
           2012: '11',
           2013: '10',
           2014: '9',
           2015: '27',
           2016: '26',
           2017: '38',
           2018: '0',
           2019: '1',
           2020: '2',
           2021: '3',
           2022: '4',
           2023: '5',
           2024: '6',
           2025: '7'}

# Just getting 2023 and 2024 data
dc = []
for y in range(2023,2025):
    qp = {'base':f'https://maps2.dcgis.dc.gov/dcgis/rest/services/FEEDS/MPD/MapServer/{year_id[y]}/query',
                  'params': {'outFields':"*",'where':'1=1'}}
    dc.append(query.query_esri(**qp))

dc_df = pd.concat(dc)
dc_df

,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,WARD,ANC,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,geometry
0,23204296,1702773215000,EVENING,OTHERS,MOTOR VEHICLE THEFT,3000 - 3099 BLOCK OF 30TH STREET SE,403074.149121,131528.840855,8,8B,...,007408,Precinct 115,38.851561,-76.964585,None,1.702767e+12,NaN,795161223,None,POINT (-76.96459 38.85157)
1,23204915,1702886273000,MIDNIGHT,OTHERS,THEFT/OTHER,3600 - 3699 BLOCK OF 14TH STREET NW,397162.310000,140976.590000,1,1D,...,002801,Precinct 41,38.936671,-77.032730,None,1.702882e+12,1.702882e+12,795161224,None,POINT (-77.03273 38.93668)
2,23205645,1703013494000,DAY,OTHERS,THEFT/OTHER,3100 - 3299 BLOCK OF 14TH STREET NW,397162.060000,140182.430000,1,1A,...,002802,Precinct 39,38.929517,-77.032730,None,1.703009e+12,1.703012e+12,795161225,None,POINT (-77.03273 38.92952)
3,23205869,1703037057000,EVENING,GUN,ROBBERY,4330 - 4350 BLOCK OF TEXAS AVENUE SE,405023.360000,135003.550000,7,7F,...,007703,Precinct 103,38.882854,-76.942104,None,1.703033e+12,NaN,795161226,None,POINT (-76.94211 38.88286)
4,23206310,1703109615000,EVENING,OTHERS,THEFT/OTHER,1900 - 1999 BLOCK OF MASSACHUSETTS AVENUE SE,402038.470000,135155.140000,7,7F,...,006804,Precinct 80,38.884232,-76.976505,None,1.702969e+12,1.702973e+12,795161227,None,POINT (-76.97651 38.88424)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29281,24038807,1710442081000,DAY,OTHERS,THEFT/OTHER,14TH STREET NW AND RHODE ISLAND AVENUE NW,397228.939995,137886.850013,2,2F,...,005203,Precinct 16,38.908838,-77.031949,None,1.710440e+12,1.710441e+12,795179036,None,POINT (-77.03195 38.90885)
29282,24068903,1715172588000,DAY,OTHERS,MOTOR VEHICLE THEFT,2700 - 2799 BLOCK OF HAMLIN STREET NE,402736.740000,139982.710000,5,5C,...,011100,Precinct 71,38.927718,-76.968438,None,1.715169e+12,NaN,795179039,None,POINT (-76.96844 38.92773)
29283,24121500,1723135466000,DAY,OTHERS,ROBBERY,1200 - 1299 BLOCK OF GOOD HOPE ROAD SE,401055.117631,133271.606937,8,8A,...,007503,Precinct 114,38.867266,-76.987842,ANACOSTIA,1.723132e+12,1.723136e+12,795179041,None,POINT (-76.98784 38.86727)
29284,24183109,1732539092000,DAY,OTHERS,MOTOR VEHICLE THEFT,100 - 199 BLOCK OF 34TH STREET SE,403685.846465,135593.885147,7,7F,...,007708,Precinct 132,38.888179,-76.957516,None,1.732536e+12,1.732864e+12,795179042,None,POINT (-76.95752 38.88819)


In [3]:
# This shows getting aggregate ESRI query using requests directly
# see https://andrewpwheeler.com/2025/04/14/some-notes-on-esri-server-apis/

out_stats = """[{onStatisticField: "case_number",
  outStatisticFieldName: "CrimeCount",
  statisticType: "count"},
 {onStatisticField: "latitude",
  outStatisticFieldName: "latitude",
  statisticType: "avg"},
 {onStatisticField: "longitude",
  outStatisticFieldName: "longitude",
  statisticType: "avg"},
 {onStatisticField: "CASE WHEN Crime_Category = 'ASSAULT' THEN 1 ELSE 0 END",
  outStatisticFieldName: "TotalAssault",
  statisticType: "sum",
  }
]
"""

base = "https://services.arcgis.com/v400IkDOw1ad7Yad/ArcGIS/rest/services/Daily_Police_Incidents/FeatureServer/0/query"
params = {"where": "reported_block_address NOT IN (' ') AND Crime_Category NOT IN ('MISCELLANEOUS','ALL OTHER OFFENSES')",
          "orderByFields": "crimecount DESC",
          "groupByFieldsForStatistics": "reported_block_address",
          "outStatistics": out_stats,
          "resultRecordCount": 100,
          "cacheHint": "false",
          "sqlFormat": "standard",
          "f": "json"}

# It is a bit obnoxious, using 
#res = requests.get(base,params)
# does not quite work directly for this example
# creating the big url does though
url = ('https://services.arcgis.com/v400IkDOw1ad7Yad/ArcGIS/rest/services/Police_Incidents/FeatureServer/0/query?'
       'where=reported_block_address+NOT+IN+%28%27+%27%29+AND+Crime_Category+NOT+IN+%28%27MISCELLANEOUS%27%2C%27ALL+OTHER+OFFENSES%27%29'
       '&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects'
       '&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields='
       '&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset='
       '&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false'
       '&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false'
       '&returnDistinctValues=false&cacheHint=false&orderByFields=CrimeCount+DESC'
       '&groupByFieldsForStatistics=reported_block_address'
       '&outStatistics=%5B%7BonStatisticField%3A+%22case_number%22%2C%0D%0A++outStatisticFieldName%3A+%22CrimeCount%22%2C%0D%0A++statisticType%3A+%22count%22%7D%2C%0D%0A+%7BonStatisticField%3A+%22latitude%22%2C%0D%0A++outStatisticFieldName%3A+%22latitude%22%2C%0D%0A++statisticType%3A+%22avg%22%7D%2C%0D%0A+%7BonStatisticField%3A+%22longitude%22%2C%0D%0A++outStatisticFieldName%3A+%22longitude%22%2C%0D%0A++statisticType%3A+%22avg%22%7D%2C%0D%0A+%7BonStatisticField%3A+%22CASE+WHEN+Crime_Category+%3D+%27ASSAULT%27+THEN+1+ELSE+0+END%22%2C%0D%0A++outStatisticFieldName%3A+%22TotalAssault%22%2C%0D%0A++statisticType%3A+%22sum%22%2C%0D%0A++%7D%0D%0A%5D'
       '&having=&resultOffset=&resultRecordCount=100&returnZ=false&returnM=false&returnExceededLimitFeatures=true'
       '&quantizationParameters=&sqlFormat=standard&f=json')

res = requests.get(url)
rj = res.json()
da = [v['attributes'] for v in rj['features']]
data = pd.DataFrame(da)
data

,CrimeCount,latitude,longitude,TotalAssault,reported_block_address
0,2279,22.669110,-49.825042,656,200 BLOCK S BLOUNT ST
1,2232,26.299444,-57.859119,262,2500 BLOCK S SAUNDERS ST
2,1539,25.075314,-55.029340,128,4400 BLOCK NEW BERN AVE
3,1350,26.793301,-58.805562,245,2600 BLOCK APPLIANCE CT
4,1274,34.443916,-75.562077,11,10000 BLOCK GLENWOOD AV
...,...,...,...,...,...
95,343,23.780628,-52.273380,93,400 BLOCK FAYETTEVILLE ST
96,341,35.867358,-78.568863,9,7900 BLOCK OLD WAKE FOREST RD
97,339,26.163617,-57.528077,44,1300 BLOCK S BLOUNT ST
98,338,19.473586,-42.808929,62,100 BLOCK E SOUTH ST


In [4]:
# Sometimes can scrape data from Tableau dashboards even
# Nashville guns stolen

from tableauscraper import TableauScraper as TS
url = "https://policepublicdata.nashville.gov/t/Police/views/GunsReportedStolen/DashGunMap"
ts = TS()
ts.loads(url)
workbook = ts.getWorkbook()

# workbook.getWorksheetNames()
map_data = workbook.getCsvData('Gun Map')
map_data['Date'] = pd.to_datetime(map_data['Dt'])
map_data['Date'].describe()
map_data.head()

,Block Address,c_title,Cat1,Cat3,Category,Dt,Rpt,Txt1,Zip,rn,...,Yrwk,Zone,Cnt,Latitude,Longitude,Num1,Num2,Num3,Weeks Ago,Date
0,1300 Block Massman Dr,NaN,NaN,Firearm Stolen from Vehicle,Firearm Stolen from Vehicle,9/1/2023,20230519106,NaN,37217,1,...,202335,531,1,36.13109,-86.70020,NaN,NaN,NaN,106,2023-09-01
1,S 8th St & Woodland St,NaN,NaN,Firearm Stolen from Vehicle,Firearm Stolen from Vehicle,9/3/2023,20230522467,NaN,37206,9,...,202336,211,1,36.17417,-86.75851,NaN,NaN,NaN,105,2023-09-03
2,2400 Block Music Valley Dr,NaN,NaN,Firearm Stolen from Vehicle,Firearm Stolen from Vehicle,9/4/2023,20230525208,NaN,37214,17,...,202336,515,1,36.21816,-86.69589,NaN,NaN,NaN,105,2023-09-04
3,1200 Block 5th Ave N,NaN,NaN,Firearm Stolen from Building,Firearm Stolen from Building,9/7/2023,20230532258,NaN,37208,25,...,202336,615,1,36.17547,-86.78806,NaN,NaN,NaN,105,2023-09-07
4,600 Block N Dupont Av,NaN,NaN,Firearm Stolen from Vehicle,Firearm Stolen from Vehicle,9/9/2023,20230536219,NaN,37115,33,...,202336,715,1,36.26719,-86.69728,NaN,NaN,NaN,105,2023-09-09
